# FunctionGemma Fine-tune với Unsloth

Train AI quyết định: `plant(plant_type, row, col)` hoặc `wait()`

**Ưu điểm Unsloth:**
- Nhanh hơn 2-5x so với HuggingFace
- Ít VRAM hơn (chạy được trên T4 free)
- Hỗ trợ FunctionGemma 270M native

**Output:** PyTorch model (convert OpenVINO trên local)

## 1. Cài đặt Unsloth

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

## 2. Upload Training Data

In [ ]:
from google.colab import files
import json

print("Upload training_data.json...")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
with open(filename, 'r') as f:
    raw_data = json.load(f)

print(f"\n✓ Loaded {len(raw_data)} samples")
stats = {}
for s in raw_data:
    stats[s['action']] = stats.get(s['action'], 0) + 1
print(f"  Actions: {stats}")

## 3. Load FunctionGemma với Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/functiongemma-270m-it",
    max_seq_length=max_seq_length,
    load_in_4bit=False,   # Full precision cho model nhỏ
    load_in_16bit=True,   # 16bit LoRA
    full_finetuning=True, # Full finetune vì model nhỏ
)

print(f"✓ Model loaded: {model.config._name_or_path}")

## 4. Define Tools

In [ ]:
def plant(plant_type: str, row: int, col: int):
    """
    Plant a plant at grid position.

    Args:
        plant_type: Type of plant (pea_shooter, sunflower, wall_nut, cherry_bomb, etc)
        row: Row index 0-4 (0=top, 4=bottom)
        col: Column index 0-8 (0=left, 8=right)

    Returns:
        result: Action result
    """
    return {"result": "planted"}

def wait():
    """
    Wait and do nothing. Use when seed is on cooldown or no good action available.

    Returns:
        result: Action result
    """
    return {"result": "waiting"}

TOOLS = [plant, wait]
print("✓ Tools defined:", [f.__name__ for f in TOOLS])

## 5. Prepare Dataset (với Thinking Block)

In [ ]:
from datasets import Dataset
import random

SYSTEM_MSG = """PvZ bot. Analyze game state and choose action.
- PLANTS: planted plants (type,row,col)
- ZOMBIES: zombies (type,row,col)
- SEEDS: seed packets (type,status)"""

def format_for_training(sample):
    """Format sample với <think> block + function response theo Unsloth docs"""
    action = sample["action"]
    args = sample.get("arguments", {})
    thinking = sample.get("thinking", "")

    if action == "plant":
        tool_call = {"type": "function", "function": {"name": "plant", "arguments": args}}
        func_response = {"name": "plant", "response": {"result": "planted"}}
    else:
        tool_call = {"type": "function", "function": {"name": "wait", "arguments": {}}}
        func_response = {"name": "wait", "response": {"result": "waiting"}}

    messages = [
        {"role": "developer", "content": SYSTEM_MSG},
        {"role": "user", "content": sample["game_state"]},
        {"role": "assistant", "tool_calls": [tool_call]},
        {"role": "tool", "content": [func_response]},  # Function response
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tools=TOOLS,
        tokenize=False,
        add_generation_prompt=False
    )

    # Inject <think> block TRƯỚC function call (theo Unsloth docs)
    if thinking:
        model_turn = "<start_of_turn>model\n"
        if model_turn in text:
            think_block = f"<think>\n{thinking}\n</think>\n"
            text = text.replace(model_turn, model_turn + think_block, 1)  # Chỉ replace lần đầu

    return {"text": text}

# Check if data has thinking field
has_thinking = any("thinking" in s for s in raw_data)
print(f"Dataset has thinking: {has_thinking}")

# Shuffle và format
random.shuffle(raw_data)
dataset = Dataset.from_list(raw_data)
dataset = dataset.map(format_for_training, remove_columns=dataset.features)

# Split train/test
dataset = dataset.train_test_split(test_size=0.1, shuffle=True)
print(f"✓ Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")
print(f"\nSample:\n{dataset['train'][0]['text'][:800]}...")

## 6. Training với Unsloth

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

num_samples = len(raw_data)
# Tự động điều chỉnh epochs dựa trên số samples
epochs = max(3, min(20, 500 // num_samples))

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        output_dir="pvz_gemma",
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        num_train_epochs=epochs,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        eval_strategy="epoch",
        save_strategy="epoch",
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
    ),
)

print(f"Training {num_samples} samples for {epochs} epochs...")
print(f"Batch size: 4 x 2 = 8 effective")

trainer_stats = trainer.train()
print(f"\n✓ Training complete in {trainer_stats.metrics['train_runtime']:.1f}s")

## 7. Test Model

In [ ]:
import re

def extract_tool_call(text):
    """Parse FunctionGemma output"""
    match = re.search(r"<start_function_call>call:(\w+)\{(.*?)\}<end_function_call>", text, re.DOTALL)
    if not match:
        return None
    name = match.group(1)
    args_str = match.group(2)
    args = {}
    for k, v in re.findall(r"(\w+):([^,}]+)", args_str):
        v = v.strip().replace("<escape>", "")
        try:
            args[k] = int(v)
        except:
            args[k] = v
    return {"name": name, "arguments": args}

def test_bot(game_state):
    messages = [
        {"role": "developer", "content": SYSTEM_MSG},
        {"role": "user", "content": game_state},
    ]
    inputs = tokenizer.apply_chat_template(
        messages, tools=TOOLS, add_generation_prompt=True,
        return_dict=True, return_tensors="pt"
    )
    out = model.generate(
        **inputs.to(model.device),
        max_new_tokens=64,
        top_k=64, top_p=0.95, temperature=1.0,
        pad_token_id=tokenizer.eos_token_id
    )
    output = tokenizer.decode(out[0][len(inputs["input_ids"][0]):], skip_special_tokens=False)
    return extract_tool_call(output)

print("="*50)
print("TEST PVZ BOT")
print("="*50)

test_cases = [
    "PLANTS:[]. ZOMBIES:[]. SEEDS:[(pea_shooter,ready)]",
    "PLANTS:[(pea_shooter,2,0)]. ZOMBIES:[(zombie,2,7)]. SEEDS:[(pea_shooter,cooldown)]",
    "PLANTS:[(pea_shooter,2,0)]. ZOMBIES:[(zombie,1,6)]. SEEDS:[(pea_shooter,ready)]",
    "PLANTS:[]. ZOMBIES:[(zombie,0,8),(zombie,4,7)]. SEEDS:[(pea_shooter,ready),(sunflower,ready)]",
]

for t in test_cases:
    result = test_bot(t)
    print(f"\n📥 {t}")
    print(f"📤 {result}")

## 8. Save & Download PyTorch Model (FP16)

In [ ]:
# Save model ở FP16 (giữ nguyên chất lượng, size ~536MB thay vì ~1GB)
print("Saving model (FP16)...")

# Full finetune không dùng LoRA nên dùng save_pretrained
model.save_pretrained("pvz_gemma_merged")
tokenizer.save_pretrained("pvz_gemma_merged")

# Convert sang FP16 để giảm size
import torch
from pathlib import Path

model_path = Path("pvz_gemma_merged")

# Convert safetensors files
for sf_file in model_path.glob("*.safetensors"):
    print(f"Converting {sf_file.name} to FP16...")
    from safetensors.torch import load_file, save_file
    tensors = load_file(sf_file)
    tensors = {k: v.half() if v.dtype == torch.float32 else v for k, v in tensors.items()}
    save_file(tensors, sf_file)

print("✓ Model saved to pvz_gemma_merged/")

# Zip và download
!zip -r pvz_gemma_merged.zip pvz_gemma_merged/
print("\n✓ Ready for download: pvz_gemma_merged.zip")
!ls -lh pvz_gemma_merged.zip

In [ ]:
from google.colab import files
files.download('pvz_gemma_merged.zip')

## (Optional) Push to HuggingFace Hub

In [ ]:
# Uncomment để push lên HuggingFace
# from huggingface_hub import login
# login()
# model.push_to_hub("your-username/pvz-gemma-bot")
# tokenizer.push_to_hub("your-username/pvz-gemma-bot")